In [2]:
import pandas as pd

from sqlalchemy.engine import create_engine
import warnings
warnings.filterwarnings('ignore', message='Cannot create BigQuery Storage client*')
engine = create_engine('bigquery://', credentials_path= '../key/iowa-liquor-sales-365322-12ff7a5498e6.json')

# Introduction:  

This independent exploratory analysis uses public data on liquor sold in Iowa and US census population data to examine sales trends. This analysis focuses on the ten most populous Iowa counties and the time period 2018-2021. 

The ten most populous Iowa counties (out of 100 total counties) comprise 52.6% of the adult (18+) population and represent 63% of the state's total liquor sales (by volume).

## Public Datasets

Iowa Liquor Sales dataset, Iowa Department of Commerce, Alcoholic Beverages Division
`bigquery-public-data.iowa_liquor_sales`  
Data represents sales to Iowa Class "E" liquor licensees. Examples include grocery stores, liquor stores etc. which sell liquor for off-premises consumption. Unaggregated dataset stored in BigQuery.


Population data, United States Census Bureau, https://www.census.gov/quickfacts/fact/table/IA/POP010220
Accurate population numbers are limited to 2020, the year of the US census. In this analysis, population counts and percent minors for 2020 are used for all years.

## Import Census Data
#### Data taken directly from US census website. No additional cleaning performed. 

In [8]:
census_data_df = pd.read_csv('../data/census_data.csv')
census_data_df

,county,population,percent_minors
0,POLK,492401,0.245
1,LINN,230299,0.228
2,SCOTT,174669,0.235
3,JOHNSON,152854,0.197
4,BLACK HAWK,131144,0.220
5,WOODBURY,105941,0.262
6,DUBUQUE,99266,0.227
7,STORY,98537,0.164
8,DALLAS,99678,0.269
9,POTTAWATTAMIE,93667,0.234


# Cleaning Liquor Sales Data

## Duplicate Check

In [11]:
#Check all rows of unaggregated dataset for duplicate invoice number
duplicate_check_sql_statement = """
SELECT COUNT(DISTINCT invoice_and_item_number) AS distinct_rows, COUNT(*) AS total_rows
FROM `bigquery-public-data.iowa_liquor_sales.sales`
""".strip()
duplicate_check_sql_statement= pd.read_sql_query(duplicate_check_sql_statement, engine)


Distinct rows equal total rows showing that all rows are unique. 

In [13]:
county_names_check_sql_statement = """SELECT COUNT(DISTINCT county) AS county_count
FROM `bigquery-public-data.iowa_liquor_sales.sales`"""
county_names_check = pd.read_sql_query(county_names_check_sql_statement, engine)
county_names_check

,county_count
0,100


The dataset has 100 distinct counties, which is correct. Iowa has 100 counties. 

## Null check

In [16]:
#Null check on Iowa Liquor Sales dataset column: volume_sold_liters
null_check_liters_sold_sql_statement = """
SELECT 
  countif(volume_sold_liters is null) / count(1) * 100 AS percent_null_liters_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
percent_null_liters_sold = pd.read_sql_query(null_check_liters_sold_sql_statement, engine)
percent_null_liters_sold

,percent_null_liters_sold
0,0.0


In [17]:
range_liters_sold_sql_statement = """
SELECT 
  MIN(volume_sold_liters) AS min_liters_sold,
  MAX(volume_sold_liters) AS max_liters_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
range_liters_sold = pd.read_sql_query(range_liters_sold_sql_statement, engine)
range_liters_sold

,min_liters_sold,max_liters_sold
0,-648.0,13200.0


Negative values in the volume_liters_sold column are unexpected. Lets look at some of the rows with negative volume_liter_sold values and see if we find any clues. 

In [19]:
neg_values_exploration_sql_statement = """ SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023) AND volume_sold_liters < 0
LIMIT 10
  """.strip()
neg_values_exploration = pd.read_sql_query(neg_values_exploration_sql_statement, engine)
neg_values_exploration

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,RINV-04350700069,2022-11-01,2650,HY-VEE WINE AND SPIRITS / HARLAN,1808 23RD ST,HARLAN,51537.0,POINT(-95.33989 41.65056),None,SHELBY,...,11777,BLACK VELVET,12,1000,6.63,9.95,-24,-238.80,-24.0,-6.34
1,RINV-04414500125,2022-12-07,2613,HY-VEE FOOD STORE #1 / COUNCIL BLUFFS,2323 W BROADWAY,COUNCIL BLUFFS,51501.0,POINT(-95.87957 41.26193),None,POTTAWATTAMIE,...,85980,DOUGH BALL COOKIE DOUGH WHISKEY,6,750,17.54,26.31,-30,-789.30,-22.5,-5.94
2,RINV-04390900030,2022-11-28,2478,PRAIRIE MEADOWS,ONE PRAIRIE MEADOWS DRIVE,ALTOONA,50009.0,POINT(-93.49319 41.6548),None,POLK,...,19477,MAKERS MARK,12,1000,22.50,33.75,-12,-405.00,-12.0,-3.17
3,RINV-04955200070,2023-11-13,6144,NORTHSIDE LIQUOR AND GROCERY / MASON CITY,1303 N FEDERAL,MASON CITY,50401.0,POINT(-93.200731023 43.163723016),None,CERRO GORDO,...,33717,PARAMOUNT SLOE GIN,12,1000,5.69,8.54,-12,-102.48,-12.0,-3.17
4,RINV-04639900030,2023-05-12,3494,SAM'S CLUB 6514 / WATERLOO,210 EAST TOWER PARK DR,WATERLOO,50702.0,POINT(-92.352287039 42.4544050040001),None,BLACK HAWK,...,86112,DEKUYPER TRIPLE SEC,12,1000,2.33,3.50,-12,-42.00,-12.0,-3.17
5,RINV-04881000086,2023-10-04,5916,ANOTHER ROUND / DEWITT,622 S 6TH AVE,DEWITT,52742.0,POINT(-90.538996227 41.809630534),None,CLINTON,...,86251,JUAREZ TRIPLE SEC,12,1000,2.50,3.75,-12,-45.00,-12.0,-3.17
6,RINV-04596500092,2023-04-13,3773,BENZ DISTRIBUTING,501 7TH AVE SE,CEDAR RAPIDS,52401.0,POINT(-91.659639837 41.9755132650001),None,LINN,...,86507,PARAMOUNT TRIPLE SEC,12,1000,4.09,6.14,-12,-73.68,-12.0,-3.17
7,RINV-04915400094,2023-10-23,5916,ANOTHER ROUND / DEWITT,622 S 6TH AVE,DEWITT,52742.0,POINT(-90.538996227 41.809630534),None,CLINTON,...,89387,JUAREZ GOLD,12,1000,8.50,12.75,-12,-153.00,-12.0,-3.17
8,RINV-04630300025,2023-05-05,3549,QUICKER LIQUOR STORE,1414 48TH ST,FORT MADISON,52627.0,POINT(-91.372709019 40.624260996),None,LEE,...,10809,CROWN ROYAL REGAL APPLE,6,1750,33.99,50.99,-6,-305.94,-10.5,-2.77
9,RINV-04730000131,2023-07-05,2191,KEOKUK SPIRITS,1013 MAIN,KEOKUK,52632.0,POINT(-91.387520009 40.400196002),None,LEE,...,36875,BLUE OX VODKA,6,1750,7.53,11.30,-6,-67.80,-10.5,-2.77


Rows with negative values in the volume_sold_liters column also have negative values in the bottles_sold, sale_dollars		volume_sold_gallons, suggesting that these values are legitimate and likely represent product returns. Let's make sure this is true for all negative values in the volume_sold_liters column. 

In [21]:
neg_liters_bottles_sql_statement = """ SELECT volume_sold_liters, bottles_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023) AND volume_sold_liters < 0 AND bottles_sold > 0
  """.strip()
neg_liters_bottles = pd.read_sql_query(neg_liters_bottles_sql_statement, engine)
neg_liters_bottles

,volume_sold_liters,bottles_sold


No output shows complete overlap of negative liters sold and negative bottles sold, suggesting that the negative values are valid and likely represent product returns. 

In [23]:
#Null check on Iowa Liquor Sales dataset column: county
null_check_county_sql_statement = """SELECT 
  countif(county is null) / count(1) * 100 as percent_null_county
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
percent_null_county = pd.read_sql_query(null_check_county_sql_statement, engine)
percent_null_county

,percent_null_county
0,0.304728


The store_number column has no nulls. 

Point three percent (0.3%) of values in the county column are null. There are 2,299 distinct store numbers, only two are missing valid county name. 

Strategy: Clean county column by mapping each store number to a valid county name.

In [25]:
cleaned_null_check_county_sql_statement = """
WITH 
source_table AS (
  SELECT
    volume_sold_liters,
    date,
    EXTRACT(YEAR FROM date) AS year,
    county,
    store_number
  FROM `bigquery-public-data.iowa_liquor_sales.sales`
  WHERE EXTRACT(YEAR FROM date) IN (2017, 2018, 2019, 2020, 2021,2022, 2023)
  ),

store_county_map AS ( 
  SELECT 
    store_number,
  MAX(county) AS county
  FROM source_table
  WHERE store_number IS NOT NULL
  GROUP BY store_number
  ),

joined_table AS (
  SELECT a.volume_sold_liters, a.date, a.year, a.store_number, COALESCE(a.county, b.county) AS county
  FROM source_table a
  LEFT JOIN store_county_map b ON a.store_number = b.store_number
  ),

 null_check_county AS (
   SELECT countif(county is null) / count(1) * 100 as percent_null_county
FROM joined_table)

SELECT * FROM null_check_county
  """.strip()

cleaned_percent_null_county = pd.read_sql_query(cleaned_null_check_county_sql_statement, engine)
cleaned_percent_null_county

,percent_null_county
0,0.006698


County column nulls reduced from 0.304728% to  0.006698%

In [27]:
#Null check on Iowa Liquor Sales dataset column: liquor_type
null_check_liquor_type_sql_statement = """
SELECT 
  countif(category_name is null) / count(1) * 100 as percent_null_liquor_type
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
percent_null_liquor_type = pd.read_sql_query(null_check_liquor_type_sql_statement, engine)
percent_null_liquor_type

,percent_null_liquor_type
0,0.050096


In [28]:
distinct_categories_sql_statement = """SELECT count(DISTINCT category_name) AS category_count 
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
distinct_categories = pd.read_sql_query(distinct_categories_sql_statement, engine)
distinct_categories

,category_count
0,52


In the original dataset, 5% of rows have null values in the category_name column. There are 52 distinct categories. Let's simplify the categories and fill the nulls and 'None' values. 

We'll use a CASE statement to recategorize all products. We'll combine existing categories and use the 'item_description' column to categorize items with null values.

In [30]:
cleaned_null_check_liquor_type_sql_statement = """
WITH cleaned_table AS(
SELECT CASE 
          WHEN category_name LIKE '%AMAR%' OR category_name LIKE '%CREME%'OR category_name LIKE '%LIQUEUR%' OR category_name LIKE '%CORDIAL%' OR category_name LIKE '%ANIS%' OR category_name LIKE '%TRIPLE SEC%' OR item_description LIKE '%JAGERM%' OR item_description LIKE '%LIQUEUR%' OR item_description LIKE '%SAINTS N SINNERS APPLE PIE%' THEN 'Liqueur' 
          WHEN category_name LIKE '%WHIS%' OR category_name LIKE '%BOUR%'OR category_name LIKE '%RYE%' OR item_description LIKE '%RYE%' OR item_description LIKE '%WHIS%'OR item_description LIKE '%SCOTCH%' OR category_name LIKE '%SCOTCH%' OR item_description LIKE '%BEAM%' OR item_description LIKE '%BOUR%' OR item_description LIKE '%CROWN ROYAL%' OR item_description LIKE '%JACK DAN%'OR item_description LIKE '%EVAN WILL%'OR item_description LIKE '%MAKER%MARK%' OR item_description LIKE '%SIR WINSTON%' OR item_description LIKE '%ELIJAH%' OR item_description LIKE '%JOHNNIE WALKER%'THEN 'Whiskey'
          WHEN category_name LIKE '%VOD%' OR item_description LIKE '%VOD%'OR item_description LIKE '%SMIRN%'THEN 'Vodka'
          WHEN category_name LIKE '%BRANDI%' OR category_name LIKE '%BRANDY%' OR category_name LIKE '%SCHNAPPS%' OR item_description LIKE '%COGNAC%' OR item_description LIKE '%HENNESSY%'THEN 'Brandy'
          WHEN category_name LIKE '%RUM%' OR item_description LIKE '%RUM%' OR item_description LIKE '%CAPTAIN MOR%'THEN 'Rum'
          WHEN category_name LIKE '%TEQ%' OR category_name LIKE '%MEZC%' OR item_description LIKE '%JUAREZ%' OR item_description LIKE '%TEQU%' OR item_description LIKE '%REPOSADO%' OR item_description LIKE '%TORTILLA GOLD%' OR item_description LIKE '%MONTEZUMA%' THEN 'Tequila and Mezcal'
          WHEN category_name LIKE '%GIN%' OR item_description LIKE '%GIN%' THEN 'Gin'
          WHEN category_name LIKE '%COCKT%' OR item_description LIKE '%TARANTULA%' THEN 'Cocktails'
          ELSE 'Other' END AS liquor_type
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
)
SELECT
  countif(liquor_type is null) / count(1) * 100 as percent_null_liquor_type,
  count(DISTINCT liquor_type) AS distinct_category_count
FROM cleaned_table
  """.strip()
cleaned_percent_null_liquor_type = pd.read_sql_query(cleaned_null_check_liquor_type_sql_statement, engine)
cleaned_percent_null_liquor_type

,percent_null_liquor_type,distinct_category_count
0,0.0,9


Our cleaning and recategorization reduced nulls in the category_name column from 5% to 0% and 52 categories to nine simplified categories. We will refer to this categorization as 'liquor_type' moving forward for clarity. 

## Conclusion

In this notebook, we established that the portion of dataset representing sales from 2018 to 2023 has 29,006,010 rows with no duplicate rows. 

There are 100 distinct counties, which is consistent with Iowa's 100 counties. Three percent of the values in the county column were null. These missing values were filled using store numbers to map to the correct county. Nulls in the county column were all filled reducing percent nulls to 0%. 

The liters_sold column had 0 nulls. Negative values exist, which seemed strange, but as negative liters sold correspond exactly with negative values in the bottles_sold column, it appears that these instances represent returned orders. 

Cleaning and recategorization in the liquor category_name column eliminated nulls (5% to 0%) and simplified categories from 52 to nine. Moving forward, liquor category name will be referred to as "liquor type". 